In [1]:
!pip install agentpy
!pip install seaborn


In [2]:
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from IPython.display import HTML

#Agent Systems

## The most basic system: An agent that does nothing

In [3]:
import agentpy as ap, numpy as np

model = ap.Model()
env = ap.Grid(model, shape=(5, 5))
ag = ap.Agent(model)

model.run(steps=10, display=False)

DataDict {
'info': Dictionary with 9 keys
'reporters': DataFrame with 1 variable and 1 row
}

## Linking agent and environment:

A *system* is a tuple (Agent, environment, state transformer)

$$S=(Ag, Env, \tau),\hspace{5mm} \tau:\mathcal{R}^{Ac} \rightarrow 2^E$$









In [4]:
model = ap.Model()
env = ap.Grid(model, shape=(5, 5))
ag = ap.Agent(model)
env.add_agents([ag])

print('\n******************************\nAgents in environment: {}\n******************************\n'.format(list(env.agents)))

model.run(steps=10, display=False)


******************************
Agents in environment: [Agent (Obj 2)]
******************************



DataDict {
'info': Dictionary with 9 keys
'reporters': DataFrame with 1 variable and 1 row
}

## An agent interacting with its environment

Agents may have some internal information (state) that allows them to act accordingly. Here, the agent is a very basic entity, without state, and interacting but not really reacting.

When it reaches the floor, agent does not vontinue its movement.

In [5]:
class DummyModel(ap.Model):

    def setup(self):
        # System = (Ag, Env, T)
        self.environment = ap.Grid(self, (5, 5))
        self.agent = ap.Agent(self)
        # Adding agent to its environment
        self.environment.add_agents([self.agent], positions=[(0,0)])

#    def update(self):
#        print('updating***')

    def step(self):
        # Agent only moves down (a falling agent)
        print("*****************************\nAgent's position in environment: {}".format(self.environment.positions[self.agent]))
        self.environment.move_by(self.agent, (1, 0))


dummyModel = DummyModel()
dummyModel.run(steps=10, display=False)

*****************************
Agent's position in environment: (0, 0)
*****************************
Agent's position in environment: (1, 0)
*****************************
Agent's position in environment: (2, 0)
*****************************
Agent's position in environment: (3, 0)
*****************************
Agent's position in environment: (4, 0)
*****************************
Agent's position in environment: (4, 0)
*****************************
Agent's position in environment: (4, 0)
*****************************
Agent's position in environment: (4, 0)
*****************************
Agent's position in environment: (4, 0)
*****************************
Agent's position in environment: (4, 0)


DataDict {
'info': Dictionary with 9 keys
'reporters': DataFrame with 1 variable and 1 row
}

##Agent with state

It is possible to model an agent that is aware of its position. This can be considered as a *state* of the agent. More complex states could consider many fields such as health, energy, money, etc.

Modificactions of agent's fiels most be donde with caution. Environment may not be aware of this modification if it is not notified properly.

**Ecercise:** How can you allow this dummy agent agent to update accurately its position?

In [6]:
debug = True
import agentpy as ap

'''
A simple agent that moves in a single direction
'''
class DummyAgent(ap.Agent):

    def setup(self):
        # Agent's position. It is regarded as its state.
        self.pos = (0, 0)

    def execute(self):
        # Agent's will update its position
        self.pos = (self.pos[0] + 1, self.pos[1])


class DummyModel(ap.Model):

    def setup(self):
        # System = (Ag, Env, T)
        self.environment = ap.Grid(self, (5, 5))
        self.agent = DummyAgent(self)

        # Setting up agent's position and adding agent to its environment
        self.agent.setup()
        self.environment.add_agents([self.agent], positions=[(0,0)])


    def step(self):
        if self.p.print:
            print("********************\nAgent's state: \t\t{}\nPosition in environment:{}".format( self.agent.pos, self.environment.positions[self.agent]))
        self.environment.move_by(self.agent, (1, 0))
        self.agent.execute()

parameters = {'print': True}
dummyModel = DummyModel(parameters)
result = dummyModel.run(steps=10, display=False)


********************
Agent's state: 		(0, 0)
Position in environment:(0, 0)
********************
Agent's state: 		(1, 0)
Position in environment:(1, 0)
********************
Agent's state: 		(2, 0)
Position in environment:(2, 0)
********************
Agent's state: 		(3, 0)
Position in environment:(3, 0)
********************
Agent's state: 		(4, 0)
Position in environment:(4, 0)
********************
Agent's state: 		(5, 0)
Position in environment:(4, 0)
********************
Agent's state: 		(6, 0)
Position in environment:(4, 0)
********************
Agent's state: 		(7, 0)
Position in environment:(4, 0)
********************
Agent's state: 		(8, 0)
Position in environment:(4, 0)
********************
Agent's state: 		(9, 0)
Position in environment:(4, 0)


## Displaying agents

In [7]:
from IPython.display import HTML
from matplotlib import pyplot as plt
import seaborn as sns

def my_plot(model, ax):
    grid = np.zeros(model.environment.shape)
    print(model.environment.positions)
    for agent, pos in model.environment.positions.items():
        grid[pos] = agent.id
    #sns.heatmap(ax=ax, grid, annot=True)
    ax.imshow(grid, cmap='Greys')

fig, ax = plt.subplots()
parameters = {'print': False, 'steps':5}
dummyModel = DummyModel(parameters)
animation = ap.animate(dummyModel, fig, ax, my_plot)
HTML(animation.to_jshtml())

{DummyAgent (Obj 2): (0, 0)}
{DummyAgent (Obj 2): (0, 0)}
{DummyAgent (Obj 2): (1, 0)}
{DummyAgent (Obj 2): (2, 0)}
{DummyAgent (Obj 2): (3, 0)}
{DummyAgent (Obj 2): (4, 0)}
{DummyAgent (Obj 2): (4, 0)}


#A multiagent system

When two or more agents interact in the same environment, the system it can be considered a MultiAgent System (MAS).

$$S=\left((Ag_1, \ldots, Ag_n), Env, (\tau_1,\ldots ,\tau_n)\right)$$

In [8]:
'''
A simple agent that moves in a single direction
'''
class DummyAgent(ap.Agent):

    def setup(self, dir):
        # Agent's position. It is regarded as its state.
        self.direction = dir

    def execute(self):
        # Agent's will update its position  (if suitable, agent can keep a reference to its environment)
        self.model.environment.move_by(self, self.direction)

'''
A simple class modeling the MAS contatinin DummyAgent
'''
class DummyModel(ap.Model):

    def setup(self):
        # System = ((Ag1, Ag2), Env, ())
        # The agents in this system will only move left or right
        self.environment = ap.Grid(self, (5, 5))
        self.right_agent = DummyAgent(self, (0, 1))
        self.left_agent = DummyAgent(self, (0, -1))
        self.environment.add_agents([self.right_agent, self.left_agent], positions=[(2,0), (2,4)])


    def step(self):
        if self.p.print:
            print("********************\nRight agent positions:{}".format(self.environment.positions[self.right_agent]))
            print("Left agent positions:{}\n********************".format(self.environment.positions[self.left_agent]))
        # On each step agents move in a given direction
        self.environment.agents.execute()

fig, ax = plt.subplots()
parameters = {'print': True, 'steps':5}
dummyModel = DummyModel(parameters)
animation = ap.animate(dummyModel, fig, ax, my_plot)
HTML(animation.to_jshtml())

{DummyAgent (Obj 2): (2, 0), DummyAgent (Obj 3): (2, 4)}
{DummyAgent (Obj 2): (2, 0), DummyAgent (Obj 3): (2, 4)}
********************
Right agent positions:(2, 0)
Left agent positions:(2, 4)
********************
{DummyAgent (Obj 2): (2, 1), DummyAgent (Obj 3): (2, 3)}
********************
Right agent positions:(2, 1)
Left agent positions:(2, 3)
********************
{DummyAgent (Obj 2): (2, 2), DummyAgent (Obj 3): (2, 2)}
********************
Right agent positions:(2, 2)
Left agent positions:(2, 2)
********************
{DummyAgent (Obj 2): (2, 3), DummyAgent (Obj 3): (2, 1)}
********************
Right agent positions:(2, 3)
Left agent positions:(2, 1)
********************
{DummyAgent (Obj 2): (2, 4), DummyAgent (Obj 3): (2, 0)}
********************
Right agent positions:(2, 4)
Left agent positions:(2, 0)
********************
{DummyAgent (Obj 2): (2, 4), DummyAgent (Obj 3): (2, 0)}


##Interaction between agents
In a MAS, agents interact with each other within their area of influence. For some agents, occupying the same position in the environment is admissible, but it may be inadmissible if physical conditions do not allow this to happen.

In [9]:
import agentpy as ap

'''
A simple agent that moves in a single direction and avoids collisions
'''
class DummyAgent(ap.Agent):

    # Setting move direction
    def setup(self, dir, env):
        self.direction = dir
        self.env = env

    # Gets the agent's position directly from environment
    def get_position(self):
        return self.env.positions[self]

    #########################
    # Collision detection step:
    # Each step, agent checks if cell is available. If so, it moves
    #########################
    def execute(self):
        # Get position of cell of next movement.
        next_position = np.array(self.direction) + self.get_position()
        # Checks if no agents are placed in next_position. If ocuppied, agent does not move
        if tuple(next_position) not in self.env.positions.values():
            self.env.move_by(self, self.direction)


'''
If things start looking complex in the Model and Agent classes, it may be suitable to add
methods according to how agents transofrm its environment
'''
class DummyEnvionment(ap.Grid):

    def setup(self):
        pass


'''
A simple class modeling the MAS containing DummyAgent
'''
class DummyModel(ap.Model):

    def setup(self):
        # System = ((Ag1, Ag2), Env, ())
        # The agents in this system will only move left or right, but should not collide!!!
        self.environment = ap.Grid(self, (5, 5))
        self.right_agent = DummyAgent(self, (0, 1), self.environment)
        self.left_agent = DummyAgent(self, (0, -1), self.environment)
        self.environment.add_agents([self.right_agent, self.left_agent], positions=[(2,0), (2,4)])
        #("Right agent positions:{}".format(self.environment.positions[self.right_agent]))
        #print("Left agent positions:{}\n********************".format(self.environment.positions[self.left_agent]))

    def step(self):
        #print("\n********************")
        self.environment.agents.execute()
        #print("Right agent positions:{}".format(self.environment.positions[self.right_agent]))
        #print("Left agent positions:{}\n********************".format(self.environment.positions[self.left_agent]))

fig, ax = plt.subplots()
parameters = {'print': True, 'steps':5}
dummyModel = DummyModel(parameters)
animation = ap.animate(dummyModel, fig, ax, my_plot)
HTML(animation.to_jshtml())

{DummyAgent (Obj 2): (2, 0), DummyAgent (Obj 3): (2, 4)}
{DummyAgent (Obj 2): (2, 0), DummyAgent (Obj 3): (2, 4)}
{DummyAgent (Obj 2): (2, 1), DummyAgent (Obj 3): (2, 3)}
{DummyAgent (Obj 2): (2, 2), DummyAgent (Obj 3): (2, 3)}
{DummyAgent (Obj 2): (2, 2), DummyAgent (Obj 3): (2, 3)}
{DummyAgent (Obj 2): (2, 2), DummyAgent (Obj 3): (2, 3)}
{DummyAgent (Obj 2): (2, 2), DummyAgent (Obj 3): (2, 3)}


## Reactive agent

In [10]:
import agentpy as ap

'''
A simple agent that moves in a single direction and avoids collisions
'''
class DummyAgent(ap.Agent):

    # Setting move direction
    def setup(self, dir, env):
        self.direction = dir
        self.env = env

    # Gets the agent's position directly from environment
    def get_position(self):
        return self.env.positions[self]

    #########################
    # Collision detection step:
    # Each step, agent checks if cell is available. If so, it moves
    #########################
    def execute(self):
        # Get position of cell of next movement.
        next_position = np.array(self.direction) + self.get_position()
        # Checks if no agents are placed in next_position. If ocuppied, agent avoids collision
        if tuple(next_position) not in self.env.positions.values():
            self.env.move_by(self, self.direction)
        else:
            self.env.move_by(self, (int(np.random.choice([-1, 1])), 0))



'''
If things start looking complex in the Model and Agent classes, it may be suitable to add
methods according to how agents transofrm its environment
'''
class DummyEnvionment(ap.Grid):

    def setup(self):
        pass


'''
A simple class modeling the MAS containing DummyAgent
'''
class DummyModel(ap.Model):

    def setup(self):
        # System = ((Ag1, Ag2), Env, ())
        # The agents in this system will only move left or right, but should not collide!!!
        self.environment = ap.Grid(self, (5, 5))
        self.right_agent = DummyAgent(self, (0, 1), self.environment)
        self.left_agent = DummyAgent(self, (0, -1), self.environment)
        self.environment.add_agents([self.right_agent, self.left_agent], positions=[(2,0), (2,4)])
        print("Right agent positions:{}".format(self.environment.positions[self.right_agent]))
        print("Left agent positions:{}\n********************".format(self.environment.positions[self.left_agent]))

    def step(self):
        print("\n********************")
        self.environment.agents.execute()
        print("Right agent positions:{}".format(self.environment.positions[self.right_agent]))
        print("Left agent positions:{}\n********************".format(self.environment.positions[self.left_agent]))

fig, ax = plt.subplots()
parameters = {'print': True, 'steps':5}
dummyModel = DummyModel(parameters)
animation = ap.animate(dummyModel, fig, ax, my_plot)
HTML(animation.to_jshtml())

Right agent positions:(2, 0)
Left agent positions:(2, 4)
********************
{DummyAgent (Obj 2): (2, 0), DummyAgent (Obj 3): (2, 4)}
{DummyAgent (Obj 2): (2, 0), DummyAgent (Obj 3): (2, 4)}

********************
Right agent positions:(2, 1)
Left agent positions:(2, 3)
********************
{DummyAgent (Obj 2): (2, 1), DummyAgent (Obj 3): (2, 3)}

********************
Right agent positions:(2, 2)
Left agent positions:(1, 3)
********************
{DummyAgent (Obj 2): (2, 2), DummyAgent (Obj 3): (1, 3)}

********************
Right agent positions:(2, 3)
Left agent positions:(1, 2)
********************
{DummyAgent (Obj 2): (2, 3), DummyAgent (Obj 3): (1, 2)}

********************
Right agent positions:(2, 4)
Left agent positions:(1, 1)
********************
{DummyAgent (Obj 2): (2, 4), DummyAgent (Obj 3): (1, 1)}

********************
Right agent positions:(2, 4)
Left agent positions:(1, 0)
********************
{DummyAgent (Obj 2): (2, 4), DummyAgent (Obj 3): (1, 0)}


## Communication and negotiation

Once agents encounter each other, they may react to this conditions (for example when finding an obstacle) or in this case, try to achieve an agreement.

A first step shold be to establish a communication protocol. For example: direct communication.

In [11]:
import agentpy as ap

'''
A more complex that moves in a direction, avoids collision, communicates
and negotiates a solution to continue its direction
'''
class DummyAgent(ap.Agent):

    # Setting move direction
    def setup(self, dir, env):
        self.direction = dir
        self.env = env

    # Gets the agent's position directly from environment
    def get_position(self):
        return self.env.positions[self]

    #########################
    # Collision detection step:
    # Each step, agent checks if cell is available. If so, it moves
    #########################
    def execute(self):
        # Get position of cell of next movement.
        next_position = np.array(self.direction) + self.get_position()
        # Checks if no agents are placed in next_position. If ocuppied, agent does not move
        if (other := self.env.who_is_at_cell(tuple(next_position))) is None:
            self.env.move_by(self, self.direction)
        else:
            self.send_message(other, 'Excuse me Sir Agent {}, want to pass!'.format(other.id))

    # A direct communication channel with a specific agent
    def send_message(self, other_agent, message):
        other_agent.recieve_message(self, message)

    # A direct communication channel with a specific agent
    def recieve_message(self, other_agent, message):
        print('Message from Agent {}: "{}". Now what?'.format(other_agent.id, message))




'''
If things start looking complex in the Model and Agent classes, it may be suitable to add
methods according to how agents transofrm its environment
'''
class DummyEnvironment(ap.Grid):

    def setup(self):
        pass

    # A method that allows to retreive agents in a given position.
    # Up  now we are assuming only 1 in a given cell
    def who_is_at_cell(self, pos):
        for ag in self.agents:
            if pos == self.positions[ag]:
                return ag


'''
A simple class modeling the MAS containing DummyAgent and DummyEnvironment
'''
class DummyModel(ap.Model):

    def setup(self):
        # System = ((Ag1, Ag2), Env, ())
        # The agents in this system will only move left or right, but should not collide!!!
        self.environment = DummyEnvironment(self, (3, 6))
        self.right_agent = DummyAgent(self, (0, -1), self.environment)
        self.left_agent = DummyAgent(self, (0, 1), self.environment)
        self.environment.add_agents([self.left_agent, self.right_agent], positions=[(1,0), (1,5)])
        print("\n********************")
        print("Right agent positions:{}".format(self.environment.positions[self.right_agent]))
        print("Left agent positions:{}\n********************".format(self.environment.positions[self.left_agent]))

    def step(self):
        print("\n********************")
        self.environment.agents.execute()
        print("Right agent positions:{}".format(self.environment.positions[self.right_agent]))
        print("Left agent positions:{}\n********************".format(self.environment.positions[self.left_agent]))

fig, ax = plt.subplots()
parameters = {'print': True, 'steps':5}
dummyModel = DummyModel(parameters)
animation = ap.animate(dummyModel, fig, ax, my_plot)
HTML(animation.to_jshtml())


********************
Right agent positions:(1, 5)
Left agent positions:(1, 0)
********************
{DummyAgent (Obj 3): (1, 0), DummyAgent (Obj 2): (1, 5)}
{DummyAgent (Obj 3): (1, 0), DummyAgent (Obj 2): (1, 5)}

********************
Right agent positions:(1, 4)
Left agent positions:(1, 1)
********************
{DummyAgent (Obj 3): (1, 1), DummyAgent (Obj 2): (1, 4)}

********************
Right agent positions:(1, 3)
Left agent positions:(1, 2)
********************
{DummyAgent (Obj 3): (1, 2), DummyAgent (Obj 2): (1, 3)}

********************
Message from Agent 3: "Excuse me Sir Agent 2, want to pass!". Now what?
Message from Agent 2: "Excuse me Sir Agent 3, want to pass!". Now what?
Right agent positions:(1, 3)
Left agent positions:(1, 2)
********************
{DummyAgent (Obj 3): (1, 2), DummyAgent (Obj 2): (1, 3)}

********************
Message from Agent 3: "Excuse me Sir Agent 2, want to pass!". Now what?
Message from Agent 2: "Excuse me Sir Agent 3, want to pass!". Now what?
Right

Communication is a starting element for agreement. The communications channel allows to send information to reach agreement, for example, the result of flipping a coin. **Who is strating the communication and negotiation protocol in this example?**

In [12]:
import agentpy as ap, numpy as np

class Message:

    def __init__(self, key, content):
        self.key = key
        self.content = content


'''
A more complex that moves in a direction, avoids collision, communicates
and negotiates a solution to continue its direction
'''
class DummyAgent(ap.Agent):

    # Setting move direction
    def setup(self, dir, env):
        self.direction = dir
        self.env = env
        self.next = None

    # Gets the agent's position directly from environment
    def get_position(self):
        return np.array(self.env.positions[self])

    #########################
    # Collision detection step:
    # Each step, agent checks if cell is available. If so, it moves
    #########################
    def intent(self):
        self.next = self.get_position() + self.direction
        others = list(filter(
            lambda ag: np.array_equal(ag.next, self.next) or
            np.array_equal(ag.get_position(), self.next), self.env.neighbors(self, 2)))
        other = None if len(others) == 0 else others[0]
        # Checks if no agents want to occupy slef.next position. Negotiate
        if other is not None:
            self.negotiate(other)

    # Actual movement
    def execute(self):
        # Check if agent intends to move to the same position
        self.env.move_to(self, tuple(self.next))




    # A direct communication channel with a specific agent. Now, message has a more complex structure
    def send_message(self, other_agent, message):
        return other_agent.receive_message(self, message)

    # A direct communication channel with a specific agent. Now, message has a more complex structure
    def receive_message(self, other_agent, message):
        print('Message from {}: {}'.format(self.model.description[other_agent.id], message.content))
        print('{} flipping coin'.format(self.model.description[self.id]))

        # If message key is 0, negotiation is being initialized
        if message.key == 0:
            flip_result = np.random.rand() > .5
            answer = Message(1, flip_result)
            # If result is True,
            if flip_result:
                self.next = self.get_position() + self.direction
            # Otherwise I lose and move up or down
            else:
                self.next = self.get_position() + (int(np.random.choice([-1, 1])), 0)

        return answer


    def negotiate(self, other_agent):
        # Start protocol: Ask for a coin fliiping to decide who moves
        message = Message(0, 'Sorry I need to pass. Flip a coin?')
        flip_result = self.send_message(other_agent, message).content
        # If result is True, the other agent won, so I move up or down
        print('Answer from {}: {}'.format(self.model.description[other_agent.id],
                                          'Ok. You move' if flip_result else 'Ok. I move'))
        if flip_result:
            self.next = tuple(self.get_position() + (int(np.random.choice([-1, 1])), 0))
        # Otherwise, I continue my path
        else:
            self.next = tuple(self.get_position() + self.direction)


'''
If things start looking complex in the Model and Agent classes, it may be suitable to add
methods according to how agents transofrm its environment
'''
class DummyEnvironment(ap.Grid):

    def setup(self):
        pass

    # A method that allows to retreive agents in a given position.
    # Up  now we are assuming only 1 in a given cell
    def who_is_at_cell(self, pos):
        for ag in self.agents:
            if pos == self.positions[ag]:
                return ag


'''
A simple class modeling the MAS containing DummyAgent and DummyEnvironment
'''
class DummyModel(ap.Model):

    def setup(self):
        # System = ((Ag1, Ag2), Env, ())
        # The agents in this system will only move left or right, but should not collide!!!
        h, w = self.p.shape
        self.environment = DummyEnvironment(self, self.p.shape)
        self.right_agent = DummyAgent(self, (0, -1), self.environment)
        self.left_agent = DummyAgent(self, (0, 1), self.environment)
        self.environment.add_agents([self.left_agent, self.right_agent], positions=[(h//2,0), (h//2,w-1)])
        self.description = {self.right_agent.id: 'Right Agent', self.left_agent.id: 'Left Agent'}
        #print("\n********************")
        #print("Right agent positions:{}".format(self.environment.positions[self.right_agent]))
        #print("Left agent positions:{}\n********************".format(self.environment.positions[self.left_agent]))

    # Use update to check all agents intents
    def update(self):
        self.environment.agents.intent()

    def step(self):
        #print("\n********************")
        self.environment.agents.execute()
        #print("Right agent positions:{}".format(self.environment.positions[self.right_agent]))
        #print("Left agent positions:{}\n********************".format(self.environment.positions[self.left_agent]))



fig, ax = plt.subplots()
parameters = {'print': True, 'steps':7, 'shape':(3,5)}
dummyModel = DummyModel(parameters)
animation = ap.animate(dummyModel, fig, ax, my_plot)
HTML(animation.to_jshtml())

{DummyAgent (Obj 3): (1, 0), DummyAgent (Obj 2): (1, 4)}
{DummyAgent (Obj 3): (1, 0), DummyAgent (Obj 2): (1, 4)}
Message from Right Agent: Sorry I need to pass. Flip a coin?
Left Agent flipping coin
Answer from Left Agent: Ok. You move
{DummyAgent (Obj 3): (np.int64(1), np.int64(1)), DummyAgent (Obj 2): (np.int64(1), np.int64(3))}
{DummyAgent (Obj 3): (np.int64(1), np.int64(2)), DummyAgent (Obj 2): (np.int64(0), np.int64(3))}
{DummyAgent (Obj 3): (np.int64(1), np.int64(3)), DummyAgent (Obj 2): (np.int64(0), np.int64(2))}
{DummyAgent (Obj 3): (np.int64(1), np.int64(4)), DummyAgent (Obj 2): (np.int64(0), np.int64(1))}
{DummyAgent (Obj 3): (np.int64(1), 4), DummyAgent (Obj 2): (np.int64(0), np.int64(0))}
{DummyAgent (Obj 3): (np.int64(1), 4), DummyAgent (Obj 2): (np.int64(0), 0)}
{DummyAgent (Obj 3): (np.int64(1), 4), DummyAgent (Obj 2): (np.int64(0), 0)}


## Visualizing agents